In [52]:
#effigy sync process

import os
from datetime import datetime
from dateutil.parser import parse
import time
from PIL import Image 
import glob
import ntpath

import mysql.connector
from mysql.connector import errorcode
import logging


logging.basicConfig(level=logging.DEBUG, filename='effigySync2.log', format='%(asctime)s %(name)s %(levelname)s:%(message)s')
logger = logging.getLogger(__name__)


In [53]:

#
SOURCEPATH='Y:/server/EFFIGY_PICS'
#SOURCEPATH='Y:/server/EFFIGY_PICS2'
#SOURCEPATH = 'C:/Users/user/Pictures'
MISSING=''
MISSING_0='0'
MISSING_DT='2100-01-01 0:0:0'
MIN_IMAGE_SIZE = 4000

In [54]:
def ConvertListToDic(lst): 
    res_dct = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)} 
    return res_dct 

In [55]:
def fixSlashes(val):
    val = val.replace('\\', '/')
    return val   


In [56]:
def getDt():
    now = datetime.now()
    dt = now.strftime('%Y-%m-%d %H:%M:%S')
    return(dt)


In [57]:
getDt()

'2020-04-16 19:36:15'

In [58]:
def  getFileCreationTime(file):     
    d =datetime.utcfromtimestamp(os.path.getctime(file)).strftime("%Y/%m/%d %H:%M:%S")
    return(d)


In [59]:
getFileCreationTime('Y:/server/EFFIGY_PICS2/Zdjecia/DCP_0474.JPG')

'2020/04/16 18:16:55'

In [60]:
def  getFileModificationTime(file):     
    d =datetime.utcfromtimestamp(os.path.getmtime(file)).strftime("%Y/%m/%d %H:%M:%S")
    return(d)


In [61]:
getFileModificationTime('Y:/server/EFFIGY_PICS2/Zdjecia/DCP_0474.JPG')

'1999/09/19 02:21:58'

In [62]:
#cust_isDate(dt)
def custom_is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

In [63]:

a=set()
#folders = []
# r=root, d=directories, f = files
for r, d, f in os.walk(SOURCEPATH):       
    for f1 in f:
        ext = os.path.splitext(f1)[1]
        a.add(ext) 

In [64]:
def createThumbnailss(srcPath,destPath,size=120):
    for infile in glob.glob(srcPath):
        print("infile",infile)
        file, ext = os.path.splitext(infile)
        print(file,ext)
        print(destPath)
        im = Image.open(infile)
        im.thumbnail(size,size)
        im.save(destPath + "/thumbnail1.png", "PNG")

In [65]:
def createThumbnails(srcFile, destPath, basewidth = 30):
    img = Image.open(srcFile)
    file = ntpath.basename(srcFile)

    wpercent = (basewidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((basewidth,hsize), Image.ANTIALIAS)
    destFile =  "./"+ file
    img.save(destFile)
    

In [66]:
createThumbnails("Y:\server\EFFIGY_PICS2\Zdjecia\other/IMG_0132.jpg","c:/temp",)

In [67]:
def getID(tbl,thisid, key,val):
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "select " + thisid + " from " +  tbl  + " where " + key + "= %s"
        #print (ssql)
        mycursor.execute(ssql, (val,))
        rows = mycursor.fetchall()
        columns = [desc[0] for desc in mycursor.description]
        newid = []
        for row in rows:
            # we dont need json or column names only one row returned
            #row = dict(zip(columns, row))
            newid.append(row)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"+val
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "getID - Completed OK!:" + str(val)
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("getID DB Closed"+ str(val))
        # obejct is a tuple we retrieve only id
        print(newid)
        if len(newid) >= 1:
            id = newid[0][0]
        else:
            id = -1
        return (id)

In [68]:
def getImageID(tbl,thisid, key,val,key2,val2):
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "select " + thisid + " from " +  tbl  + " where " + key + "= %s and  " + key2 + "= %s"
        #print (ssql)
        mycursor.execute(ssql, (fixSlashes(val),val2,))
        rows = mycursor.fetchall()
        columns = [desc[0] for desc in mycursor.description]
        newid = []
        for row in rows:
            # we dont need json or column names only one row returned
            #row = dict(zip(columns, row))
            newid.append(row)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"+val
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "getID - Completed OK!:" + val
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("getID DB Closed"+ val)
        # obejct is a tuple we retrieve only id
        print(newid)
        if len(newid) >= 1:
            id = newid[0][0]
        else:
            id = -1
        return (id)

In [69]:
newid= getImageID("media","media_id", "path","c:\\","name","test.jpg")

[]


In [70]:
def getValidExtensions():
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "select extension from supported_ext order by 1"
        mycursor.execute(ssql)
        rows = mycursor.fetchall()
        columns = [desc[0] for desc in mycursor.description]
        rec = []
        for row in rows:
            # we dont need json or column names only one row returned
            #row = dict(zip(columns, row))
            rec.append(row)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"+val
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "getID - Completed OK!:"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("getID DB Closed")
        # converting it to a SET, fast lookup
        ext_set=set()  
        for i in rec:
            ext_set.add (i[0])
        return (ext_set)

In [71]:
def getValidMediType_Extensions():
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "select extension, media_type_id from media_type_ext order by 1"
        mycursor.execute(ssql)
        rows = mycursor.fetchall()
        columns = [desc[0] for desc in mycursor.description]
        rec = []
        for row in rows:
            # we dont need json or column names only one row returned
            #row = dict(zip(columns, row))
            rec.append(row)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"+val
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "getID - Completed OK!:"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("getID DB Closed")
        return (dict(rec))    

In [72]:
r = getValidMediType_Extensions()
r.get('.avi')

'V'

In [73]:
def getSkipFolder():
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "select skip_root from skip_root"
        mycursor.execute(ssql)
        rows = mycursor.fetchall()
        columns = [desc[0] for desc in mycursor.description]
        rec = []
        for row in rows:
            # we dont need json or column names only one row returned
            #row = dict(zip(columns, row))
            rec.append(row)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"+val
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "getID - Completed OK!:"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("getID DB Closed")
        # converting it to a SET, fast lookup
        _set=set()  
        for i in rec:
            _set.add (i[0])
        return (_set)

In [74]:
getSkipFolder()

{'Y:\\server\\EFFIGY_PICS2\\Zdjecia\\tomcat-ZDJECIA',
 'Y:\\server\\EFFIGY_PICS\\Zdjecia\\tomcat-ZDJECIA'}

In [75]:
import functools 
import operator  
def convertTuple(tup): 
    str1 =  ''.join(str(tup)) 
    return str1

In [76]:
#get valid extensions
EXT = getValidExtensions()
#  check
'avi' in EXT
EXT

{'.avi', '.jpeg', '.jpg', '.mov', '.mp3', '.mp4', '.mpg', '.png', '.wmv'}

In [77]:
def insertRoot (root):
    logger.info("Starting Request")
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "insert into loc_root values (null, %s,%s,%s)"
        root = fixSlashes(root)
        #print (root)
        mycursor.execute(ssql, (root,'',getDt(),))
        mydb.commit()
        #get newly created ID
        newid= getID("loc_root","root_id", "root",fixSlashes(root))
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "Root-Completed OK!"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("Root - DB Closed")
        return (newid)

In [78]:
getID("loc_folder","folder_id", "folder_path","C:ggg/Uffsssehhffrsggs/user/Pictures/2017-11111111111")

[]


-1

In [79]:
def insertFolder (root_id,root,folder,folderPath):
    logger.info("Starting Request")
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "insert into loc_folder values (null,%s, %s,%s,%s,%s,%s)"
        root = fixSlashes(root)
        folder = fixSlashes(folder)
        folderPath = fixSlashes(folderPath)
        #print (ssql)
        mycursor.execute(ssql, (root_id,root,folder,folderPath,'',getDt(),))
        mydb.commit()
        #get newly created ID
        newid= getID("loc_folder","folder_id", "folder_path",fixSlashes(folderPath))
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "Folder Completed OK!"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("Folder check DB Closed")
        return (newid)

In [80]:
def skipRoot(root):
    skip=False
    toSkip="Y:\server\EFFIGY_PICS\Zdjecia\\tomcat-ZDJECIA"
    if len(root) >= len(toSkip):
        if (fixSlashes(toSkip).upper() == fixSlashes(root[:len(toSkip)]).upper())  :
            skip=True
    #else:
    #    print("not matching length")
    return(skip)

In [81]:
def skipFolder(root):
    skip=False
    toSkip=getSkipFolder()
    for folder in toSkip:
        if len(root) >= len(folder):
            #print(folder)
            #print(fixSlashes(folder).upper())
            #print(fixSlashes(root[:len(folder)]).upper()) 
            if (fixSlashes(folder).upper() == fixSlashes(root[:len(folder)]).upper())  :
                skip=True
                return(skip)
    return(skip)
        


In [82]:
skipFolder("Y:\server\EFFIGY_PICS2/Zdjecia\\tomcat-ZDJECIA")

True

In [83]:
#newid = insertFolder(1, "C:/Users/user/Pictures","2017-1111", "C:ggg/Uffsssehhffrsggs/user/Pictures/2017-1221111111111")
#newid

In [84]:
def insertImage (key,file_name,media_type,folder_id,path, parent_folder_id,parent_folder,attrib_id,ext,dttaken,dtmodified,size):
    logger.info("Saving image")
    #print(key,file_name,media_type,folder_id,path, parent_folder_id,parent_folder,attrib_id,ext)
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "insert into media values (null,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        path = fixSlashes(path)
        parent_folder = fixSlashes(parent_folder)
        #print (ssql)
        mycursor.execute(ssql, ( key,file_name,media_type,folder_id,path,parent_folder_id,\
                                parent_folder,"sDesc",file_name,0,ext,size,dttaken,dtmodified,getDt(),getDt()))
        mydb.commit()
        #get newly created ID
        newid= getImageID("media","media_id", "path",path,"name",file_name)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "Folder Completed OK!"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("Folder check DB Closed")
        return (newid)

In [85]:
getDt()

'2020-04-16 19:36:42'

In [86]:
def importFoldersAndImages ():
    #get Valid extensions first
    c=0
    EXT = getValidExtensions()
    EXTTypeDic  = getValidMediType_Extensions()
    #  check
    '' in EXT
    #print(EXT)
    for r, d, f in os.walk(SOURCEPATH):
        #print("dir",d,r)
        if len(d) >0:
            if  skipRoot(r):
                print("Skipped Root:",r)
            else:
                idR = insertRoot(r)
                #print("Processed Root:",idR,r)
                for dir in d:
                    #print ("will INSERT dir:",idR,dir)
                    #print("about to insert", idR,r,dir,r + '/' + dir)
                    idF = insertFolder(idR,r,dir,r + "/"+ dir)
                    #print("Created folder",idF)
                    pth = r +"/"+dir
                    #print("path", pth)
                    files = [f for f in os.listdir(pth) if os.path.isfile(os.path.join(pth,f))]
                    for f in files:
                        file_ext= os.path.splitext(f)[1].lower()
                        key= fixSlashes(r +"/"+dir + "/" + f)
                        size = os.path.getsize(key)
                        #print("size", size,file_ext)                        
                        if (file_ext in EXT) and size > MIN_IMAGE_SIZE:                            
                            mediaType = EXTTypeDic.get(file_ext)
                            exif=getExifDate(key,mediaType)
                            dttaken = exif["DTTAKEN"]
                            dtmodified= exif["DTMODIFY"]
                            
                            #print("Processing:" + f)
                            #inserImage(file_name,media_type,folder_id,path, parent_folder_id,parent_folder,attrib_id,ext):                       
                            idM = insertImage(key,f,mediaType,idF, r +"/"+dir, idR,r,"",file_ext,dttaken,dtmodified,size)
                            #print(idM) 
                           
    return(c)                    
                    
    
                    
                    
                
        
    

In [603]:
c= importFoldersAndImages()
print ("total:",c)

[(11,)]
[(19,)]
[(20,)]
[(21,)]
[(12,)]
[(22,)]
[(23,)]
[(15,)]
[(16,)]
[(17,)]
[(18,)]
[(19,)]
[(20,)]
[(21,)]
[(13,)]
[(24,)]
[(22,)]
[(23,)]
[(24,)]
[(25,)]
[(26,)]
[(27,)]
[(28,)]
[(29,)]
[(30,)]
[(31,)]
[(32,)]
[(33,)]
[(34,)]
[(35,)]
[(36,)]
[(37,)]
[(38,)]
[(39,)]
[(40,)]
[(41,)]
[(42,)]
[(43,)]
[(44,)]
[(45,)]
[(46,)]
[(47,)]
[(48,)]
[(49,)]
[(50,)]
[(51,)]
[(52,)]
[(53,)]
[(54,)]
[(55,)]
[(56,)]
[(57,)]
[(58,)]
[(59,)]
[(60,)]
[(61,)]
[(62,)]
[(63,)]
[(64,)]
[(65,)]
[(66,)]
[(67,)]
[(68,)]
[(69,)]
[(70,)]
[(71,)]
[(72,)]
[(73,)]
[(74,)]
[(75,)]
[(76,)]
[(77,)]
[(78,)]
[(79,)]
[(80,)]
[(81,)]
[(82,)]
[(83,)]
[(84,)]
[(85,)]
[(86,)]
[(87,)]
[(88,)]
[(89,)]
[(90,)]
[(91,)]
[(92,)]
[(93,)]
[(94,)]
[(95,)]
[(96,)]
[(97,)]
[(98,)]
[(99,)]
[(100,)]
[(101,)]
[(102,)]
[(103,)]
[(104,)]
[(105,)]
[(106,)]
[(107,)]
[(108,)]
[(109,)]
[(110,)]
[(111,)]
[(112,)]
[(113,)]
[(114,)]
[(115,)]
[(116,)]
[(117,)]
[(118,)]
[(119,)]
[(120,)]
[(121,)]
[(122,)]
[(123,)]
[(124,)]
[(125,)]
[(126,)]
[(127

[(924,)]
[(925,)]
[(926,)]
[(927,)]
[(928,)]
[(929,)]
[(930,)]
[(931,)]
[(932,)]
[(933,)]
[(934,)]
[(935,)]
[(936,)]
[(937,)]
[(938,)]
[(939,)]
[(940,)]
[(941,)]
[(942,)]
[(943,)]
[(944,)]
[(945,)]
[(946,)]
[(947,)]
[(948,)]
[(949,)]
[(950,)]
[(951,)]
[(952,)]
[(953,)]
[(954,)]
[(955,)]
[(956,)]
[(957,)]
[(958,)]
[(959,)]
[(960,)]
[(961,)]
[(962,)]
[(963,)]
[(964,)]
[(965,)]
[(966,)]
[(967,)]
[(968,)]
[(969,)]
[(970,)]
[(971,)]
[(972,)]
[(973,)]
[(974,)]
[(975,)]
[(976,)]
[(977,)]
[(978,)]
[(979,)]
[(980,)]
[(981,)]
[(982,)]
[(983,)]
[(984,)]
[(985,)]
[(986,)]
[(987,)]
[(988,)]
[(989,)]
[(990,)]
[(991,)]
[(992,)]
[(993,)]
[(994,)]
[(995,)]
[(996,)]
[(997,)]
[(998,)]
[(999,)]
[(1000,)]
[(1001,)]
[(1002,)]
[(1003,)]
[(1004,)]
[(1005,)]
[(1006,)]
[(1007,)]
[(1008,)]
[(1009,)]
[(1010,)]
[(1011,)]
[(1012,)]
[(1013,)]
[(1014,)]
[(1015,)]
[(1016,)]
[(1017,)]
[(1018,)]
[(1019,)]
[(1020,)]
[(1021,)]
[(1022,)]
[(1023,)]
[(1024,)]
[(1025,)]
[(1026,)]
[(1027,)]
[(1028,)]
[(1029,)]
[(1030,)]
[(1031

[(1751,)]
[(1752,)]
[(1753,)]
[(1754,)]
[(1755,)]
[(1756,)]
[(1757,)]
[(1758,)]
[(1759,)]
[(1760,)]
[(1761,)]
[(1762,)]
[(1763,)]
[(1764,)]
[(1765,)]
[(1766,)]
[(1767,)]
[(1768,)]
[(1769,)]
[(1770,)]
[(1771,)]
[(1772,)]
[(1773,)]
[(1774,)]
[(1775,)]
[(1776,)]
[(1777,)]
[(1778,)]
[(1779,)]
[(1780,)]
[(1781,)]
[(1782,)]
[(1783,)]
[(1784,)]
[(1785,)]
[(1786,)]
[(1787,)]
[(1788,)]
[(1789,)]
[(1790,)]
[(1791,)]
[(1792,)]
[(1793,)]
[(1794,)]
[(1795,)]
[(1796,)]
[(1797,)]
[(1798,)]
[(1799,)]
[(1800,)]
[(1801,)]
[(1802,)]
[(1803,)]
[(1804,)]
[(1805,)]
[(1806,)]
[(1807,)]
[(1808,)]
[(1809,)]
[(1810,)]
[(1811,)]
[(1812,)]
[(1813,)]
[(1814,)]
[(1815,)]
[(1816,)]
[(1817,)]
[(1818,)]
[(1819,)]
[(1820,)]
[(1821,)]
[(1822,)]
[(1823,)]
[(1824,)]
[(1825,)]
[(1826,)]
[(1827,)]
[(1828,)]
[(1829,)]
[(1830,)]
[(1831,)]
[(1832,)]
[(1833,)]
[(1834,)]
[(1835,)]
[(1836,)]
[(1837,)]
[(1838,)]
[(1839,)]
[(1840,)]
[(1841,)]
[(1842,)]
[(1843,)]
[(1844,)]
[(1845,)]
[(1846,)]
[(1847,)]
[(1848,)]
[(1849,)]
[(1850,)]


[(2441,)]
[(2442,)]
[(2443,)]
[(2444,)]
[(2445,)]
[(2446,)]
[(2447,)]
[(2448,)]
[(2449,)]
[(2450,)]
[(2451,)]
[(2452,)]
[(2453,)]
[(2454,)]
[(2455,)]
[(2456,)]
[(2457,)]
[(2458,)]
[(2459,)]
[(2460,)]
[(2461,)]
[(2462,)]
[(2463,)]
[(2464,)]
[(2465,)]
[(2466,)]
[(2467,)]
[(2468,)]
[(2469,)]
[(2470,)]
[(2471,)]
[(2472,)]
[(2473,)]
[(2474,)]
[(2475,)]
[(2476,)]
[(2477,)]
[(2478,)]
[(2479,)]
[(2480,)]
[(2481,)]
[(2482,)]
[(2483,)]
[(2484,)]
[(2485,)]
[(2486,)]
[(2487,)]
[(2488,)]
[(2489,)]
[(2490,)]
[(2491,)]
[(2492,)]
[(2493,)]
[(2494,)]
[(2495,)]
[(2496,)]
[(2497,)]
[(2498,)]
[(2499,)]
[(2500,)]
[(2501,)]
[(2502,)]
[(2503,)]
[(2504,)]
[(2505,)]
[(2506,)]
[(2507,)]
[(2508,)]
[(2509,)]
[(2510,)]
[(2511,)]
[(2512,)]
[(2513,)]
[(2514,)]
[(2515,)]
[(2516,)]
[(2517,)]
[(2518,)]
[(2519,)]
[(2520,)]
[(2521,)]
[(2522,)]
[(2523,)]
[(2524,)]
[(2525,)]
[(2526,)]
[(2527,)]
[(2528,)]
[(2529,)]
[(2530,)]
[(2531,)]
[(2532,)]
[(2533,)]
[(2534,)]
[(35,)]
[(2535,)]
[(2536,)]
[(2537,)]
[(2538,)]
[(2539,)]
[(

[(3257,)]
[(3258,)]
[(3259,)]
[(3260,)]
[(3261,)]
[(3262,)]
[(3263,)]
[(3264,)]
[(3265,)]
[(3266,)]
[(3267,)]
[(3268,)]
[(3269,)]
[(3270,)]
[(3271,)]
[(3272,)]
[(41,)]
[(3273,)]
[(3274,)]
[(3275,)]
[(3276,)]
[(3277,)]
[(3278,)]
[(3279,)]
[(3280,)]
[(3281,)]
[(3282,)]
[(3283,)]
[(3284,)]
[(3285,)]
[(3286,)]
[(3287,)]
[(3288,)]
[(3289,)]
[(3290,)]
[(3291,)]
[(3292,)]
[(3293,)]
[(3294,)]
[(3295,)]
[(3296,)]
[(3297,)]
[(3298,)]
[(3299,)]
[(3300,)]
[(3301,)]
[(3302,)]
[(3303,)]
[(3304,)]
[(3305,)]
[(3306,)]
[(3307,)]
[(3308,)]
[(3309,)]
[(3310,)]
[(3311,)]
[(3312,)]
[(3313,)]
[(3314,)]
[(3315,)]
[(3316,)]
[(3317,)]
[(3318,)]
[(3319,)]
[(3320,)]
[(3321,)]
[(3322,)]
[(3323,)]
[(3324,)]
[(3325,)]
[(3326,)]
[(3327,)]
[(3328,)]
[(3329,)]
[(3330,)]
[(3331,)]
[(3332,)]
[(3333,)]
[(3334,)]
[(3335,)]
[(3336,)]
[(3337,)]
[(3338,)]
[(3339,)]
[(3340,)]
[(3341,)]
[(3342,)]
[(3343,)]
[(3344,)]
[(3345,)]
[(3346,)]
[(3347,)]
[(3348,)]
[(3349,)]
[(3350,)]
[(3351,)]
[(3352,)]
[(3353,)]
[(3354,)]
[(3355,)]
[(

...

[(19256,)]
[(19257,)]
[(19258,)]
[(19259,)]
[(19260,)]
[(19261,)]
[(19262,)]
[(19263,)]
[(19264,)]
[(19265,)]
[(19266,)]
[(19267,)]
[(19268,)]
[(19269,)]
[(19270,)]
[(19271,)]
[(19272,)]
[(19273,)]
[(19274,)]
[(19275,)]
[(19276,)]
[(19277,)]
[(19278,)]
[(19279,)]
[(19280,)]
[(19281,)]
[(19282,)]
[(19283,)]
[(19284,)]
[(19285,)]
[(19286,)]
[(19287,)]
[(19288,)]
[(19289,)]
[(19290,)]
[(19291,)]
[(19292,)]
[(19293,)]
[(19294,)]
[(19295,)]
[(19296,)]
[(19297,)]
[(19298,)]
[(19299,)]
[(19300,)]
[(19301,)]
[(19302,)]
[(19303,)]
[(19304,)]
[(19305,)]
[(19306,)]
[(19307,)]
[(19308,)]
[(19309,)]
[(19310,)]
[(19311,)]
[(187,)]
[(19312,)]
[(19313,)]
[(19314,)]
[(19315,)]
[(19316,)]
[(19317,)]
[(188,)]
[(19318,)]
[(19319,)]
[(19320,)]
[(19321,)]
[(19322,)]
[(19323,)]
[(19324,)]
[(19325,)]
[(19326,)]
[(19327,)]
[(19328,)]
[(19329,)]
[(19330,)]
[(19331,)]
[(19332,)]
[(19333,)]
[(19334,)]
[(19335,)]
[(19336,)]
[(19337,)]
[(19338,)]
[(19339,)]
[(19340,)]
[(19341,)]
[(19342,)]
[(19343,)]
[(19344,)]
[(1

[(19918,)]
[(19919,)]
[(19920,)]
[(19921,)]
[(19922,)]
[(19923,)]
[(19924,)]
[(19925,)]
[(19926,)]
[(19927,)]
[(19928,)]
[(19929,)]
[(19930,)]
[(19931,)]
[(19932,)]
[(19933,)]
[(19934,)]
[(19935,)]
[(19936,)]
[(19937,)]
[(19938,)]
[(19939,)]
[(19940,)]
[(19941,)]
[(19942,)]
[(19943,)]
[(19944,)]
[(19945,)]
[(19946,)]
[(19947,)]
[(19948,)]
[(19949,)]
[(19950,)]
[(19951,)]
[(19952,)]
[(19953,)]
[(19954,)]
[(19955,)]
[(19956,)]
[(19957,)]
[(19958,)]
[(19959,)]
[(19960,)]
[(19961,)]
[(19962,)]
[(19963,)]
[(19964,)]
[(19965,)]
[(19966,)]
[(19967,)]
[(19968,)]
[(19969,)]
[(19970,)]
[(19971,)]
[(19972,)]
[(19973,)]
[(19974,)]
[(19975,)]
[(19976,)]
[(19977,)]
[(19978,)]
[(19979,)]
[(19980,)]
[(19981,)]
[(19982,)]
[(19983,)]
[(19984,)]
[(19985,)]
[(19986,)]
[(19987,)]
[(19988,)]
[(19989,)]
[(19990,)]
[(19991,)]
[(19992,)]
[(19993,)]
[(19994,)]
[(19995,)]
[(19996,)]
[(19997,)]
[(19998,)]
[(19999,)]
[(20000,)]
[(20001,)]
[(20002,)]
[(194,)]
[(20003,)]
[(20004,)]
[(20005,)]
[(20006,)]
[(20007,)]
[

[(20663,)]
[(20664,)]
[(20665,)]
[(20666,)]
[(20667,)]
[(20668,)]
[(20669,)]
[(20670,)]
[(20671,)]
[(20672,)]
[(20673,)]
[(20674,)]
[(20675,)]
[(20676,)]
[(20677,)]
[(20678,)]
[(20679,)]
[(20680,)]
[(20681,)]
[(20682,)]
[(20683,)]
[(20684,)]
[(20685,)]
[(20686,)]
[(20687,)]
[(20688,)]
[(20689,)]
[(20690,)]
[(20691,)]
[(20692,)]
[(20693,)]
[(20694,)]
[(20695,)]
[(20696,)]
[(20697,)]
[(20698,)]
[(20699,)]
[(20700,)]
[(20701,)]
[(20702,)]
[(20703,)]
[(20704,)]
[(20705,)]
[(20706,)]
[(20707,)]
[(20708,)]
[(20709,)]
[(20710,)]
[(20711,)]
[(20712,)]
[(20713,)]
[(20714,)]
[(20715,)]
[(20716,)]
[(20717,)]
[(20718,)]
[(20719,)]
[(20720,)]
[(20721,)]
[(20722,)]
[(20723,)]
[(20724,)]
[(20725,)]
[(20726,)]
[(20727,)]
[(20728,)]
[(20729,)]
[(20730,)]
[(20731,)]
[(20732,)]
[(20733,)]
[(20734,)]
[(20735,)]
[(20736,)]
[(20737,)]
[(20738,)]
[(20739,)]
[(20740,)]
[(20741,)]
[(20742,)]
[(20743,)]
[(20744,)]
[(20745,)]
[(20746,)]
[(20747,)]
[(20748,)]
[(20749,)]
[(20750,)]
[(20751,)]
[(20752,)]
[(20753,)]

[(21407,)]
[(21408,)]
[(21409,)]
[(21410,)]
[(21411,)]
[(21412,)]
[(21413,)]
[(21414,)]
[(21415,)]
[(21416,)]
[(21417,)]
[(21418,)]
[(21419,)]
[(21420,)]
[(21421,)]
[(21422,)]
[(21423,)]
[(21424,)]
[(21425,)]
[(21426,)]
[(21427,)]
[(21428,)]
[(21429,)]
[(21430,)]
[(21431,)]
[(21432,)]
[(21433,)]
[(21434,)]
[(21435,)]
[(21436,)]
[(21437,)]
[(21438,)]
[(21439,)]
[(21440,)]
[(21441,)]
[(21442,)]
[(21443,)]
[(21444,)]
[(21445,)]
[(21446,)]
[(21447,)]
[(21448,)]
[(21449,)]
[(21450,)]
[(21451,)]
[(21452,)]
[(21453,)]
[(21454,)]
[(21455,)]
[(21456,)]
[(21457,)]
[(21458,)]
[(21459,)]
[(21460,)]
[(21461,)]
[(21462,)]
[(21463,)]
[(21464,)]
[(21465,)]
[(21466,)]
[(21467,)]
[(21468,)]
[(21469,)]
[(21470,)]
[(21471,)]
[(21472,)]
[(21473,)]
[(21474,)]
[(21475,)]
[(21476,)]
[(21477,)]
[(21478,)]
[(21479,)]
[(21480,)]
[(21481,)]
[(21482,)]
[(21483,)]
[(21484,)]
[(21485,)]
[(21486,)]
[(21487,)]
[(21488,)]
[(21489,)]
[(21490,)]
[(21491,)]
[(21492,)]
[(21493,)]
[(21494,)]
[(21495,)]
[(21496,)]
[(21497,)]

[(22770,)]
[(22771,)]
[(22772,)]
[(22773,)]
[(22774,)]
[(22775,)]
[(22776,)]
[(22777,)]
[(22778,)]
[(22779,)]
[(22780,)]
[(22781,)]
[(22782,)]
[(22783,)]
[(22784,)]
[(22785,)]
[(22786,)]
[(22787,)]
[(22788,)]
[(22789,)]
[(226,)]
[(22790,)]
[(22791,)]
[(22792,)]
[(22793,)]
[(22794,)]
[(22795,)]
[(22796,)]
[(46,)]
[(227,)]
[(22797,)]
[(22798,)]
[(22799,)]
[(22800,)]
[(22801,)]
[(22802,)]
[(22803,)]
[(22804,)]
[(22805,)]
[(22806,)]
[(22807,)]
[(22808,)]
[(22809,)]
[(22810,)]
[(22811,)]
[(22812,)]
[(22813,)]
[(22814,)]
[(22815,)]
[(22816,)]
[(22817,)]
[(22818,)]
[(22819,)]
[(22820,)]
[(22821,)]
[(22822,)]
[(22823,)]
[(22824,)]
[(22825,)]
[(22826,)]
[(22827,)]
[(22828,)]
[(22829,)]
[(22830,)]
[(22831,)]
[(22832,)]
[(47,)]
[(228,)]
[(48,)]
[(229,)]
[(22833,)]
[(22834,)]
[(22835,)]
[(22836,)]
[(22837,)]
[(22838,)]
[(49,)]
[(230,)]
[(22839,)]
[(22840,)]
[(22841,)]
[(22842,)]
[(22843,)]
[(22844,)]
[(22845,)]
[(22846,)]
[(22847,)]
[(22848,)]
[(22849,)]
[(22850,)]
[(22851,)]
[(22852,)]
[(22853,)]

In [87]:
def getImages():
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "select media_id,path, name, media_key,media_type_id from media"
        #print (ssql)
        mycursor.execute(ssql)
        rows = mycursor.fetchall()
        columns = [desc[0] for desc in mycursor.description]
        rec = []
        for row in rows:
            # we dont need json or column names only one row returned
            #row = dict(zip(columns, row))
            rec.append(row)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"+val
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "getID - Completed OK!:"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("getID DB Closed")
        _set=set()  
        for i in rec:
            _set.add (i)
        return (_set)

In [88]:
getDt()

'2020-04-16 19:36:52'

In [89]:
def getImageExifAttributes(pth,ext):
    exif= {}
    ex=""
    dic={}
    #check if image
    mdt='2090-01-01 0:0:0'
    if ext == 'P':
        # Read image       
     try:
        print(pth)
        mdt = getFileModificationTime(pth)
        cdt = getFileCreationTime(pth)
        img = Image.open(pth) 
        dic = img._getexif()
        #check if empty
     except Exception as err:
        print("issue processing" + pth)
        print(err)
        logger.error(err)
        print("WIDTH",pth)
    if  dic:
        
        if str(dic.get(40963)).isnumeric():
            exif["WIDTH"] =  str(dic.get(40963))
        else:
            exif["WIDTH"] =  str('0')
            
        exif["HEIGHT"] = str( dic.get(40962))
        exif["CAMERA"] = str( dic.get(271))
        exif["MODEL"] =  str(dic.get(272))
        exif["DTTAKEN"] = str( dic.get(36868))
        exif["DTMODIFY"] = str( dic.get(36867))
        exif["FSTOP"] =  str(dic.get(42036))
        exif["EXPOSURE"] =  str(dic.get(40960))
        exif["ORIENTATION"] = str( dic.get(274))
        exif["FLASH"] =  str(dic.get(7385))
        exif["SHUTTER"] =  str(dic.get(37377))
        exif["APERTURE"] = str( dic.get(37378))
        exif["ISOSPEED"] = str( dic.get(34855))
        exif["DIGIZOOM"] =  str(dic.get(41988))
        #print("FSTOP",exif["FSTOP"]) 
        for x in dic:
            if x != 37500 and x != 37510:
                ex= ex + (str(x) +  ":" +  str(dic.get(x)) + chr(13) +chr(10) ) 
        #limit to 5K
        exif["EXIF"] =  str(ex)[0:5000] 
    else:
        exif["WIDTH"] =  MISSING_0
        exif["HEIGHT"] = MISSING_0
        exif["CAMERA"] = MISSING
        exif["MODEL"] =  MISSING
        exif["DTTAKEN"] = MISSING_DT
        exif["DTMODIFY"] = MISSING_DT
        exif["FSTOP"] =  MISSING
        exif["EXPOSURE"] =  MISSING
        exif["ORIENTATION"] = MISSING
        exif["FLASH"] = MISSING
        exif["SHUTTER"] =  MISSING
        exif["APERTURE"] = MISSING
        exif["ISOSPEED"] = MISSING
        exif["DIGIZOOM"] =  MISSING
        exif["EXIF"] =  MISSING
        
    #overwrite mod date is wrong
    #print("From File Modify:",mdt)
    #print("From File Create:",cdt)
    #print("From exif Modify:",exif["DTMODIFY"])
    #print("From exif Create:",exif["DTTAKEN"])
    exif["DTMODIFY"]
    if exif["DTTAKEN"] == exif["DTMODIFY"]:
        if custom_is_date(mdt):
            exif["DTMODIFY"] = mdt
    return(exif)

In [90]:
def getExifDate(pth,ext):
    exif= {}
    ex=""
    dic={}
    #check if image
    mdt='2090-01-01 0:0:0'
    if ext == 'P':
        # Read image 
     try:      
        mdt = getFileModificationTime(pth)
        cdt = getFileCreationTime(pth)
        img = Image.open(pth) 
        dic = img._getexif()
        #check if empty
     except Exception as err:
        print("issue processing" + pth)
        print(err)
        logger.error(err)
        print("WIDTH",pth)
    if  dic:
        
        exif["DTTAKEN"] = str( dic.get(36868))
        exif["DTMODIFY"] = str( dic.get(36867))
         
    else:
        exif["DTTAKEN"] = MISSING_DT
        exif["DTMODIFY"] = MISSING_DT      

    if exif["DTTAKEN"] == exif["DTMODIFY"]:
        if custom_is_date(mdt):
            exif["DTMODIFY"] = mdt       
    return(exif)

In [91]:
getExifDate("Y:/server/EFFIGY_PICS2/Zdjecia/DCP_0474.JPG",'P')

{'DTTAKEN': '1999:01:31 15:39:48', 'DTMODIFY': '1999/09/19 02:21:58'}

In [92]:
getImageExifAttributes('Y:/server/EFFIGY_PICS2/Zdjecia/DCP_0474.JPG','P')

Y:/server/EFFIGY_PICS2/Zdjecia/DCP_0474.JPG


{'WIDTH': '960',
 'HEIGHT': '1280',
 'CAMERA': 'EASTMAN KODAK COMPANY',
 'MODEL': 'KODAK DC240 ZOOM DIGITAL CAMERA',
 'DTTAKEN': '1999:01:31 15:39:48',
 'DTMODIFY': '1999/09/19 02:21:58',
 'FSTOP': 'None',
 'EXPOSURE': "b'0100'",
 'ORIENTATION': '1',
 'FLASH': 'None',
 'SHUTTER': '(50, 10)',
 'APERTURE': '(35, 10)',
 'ISOSPEED': 'None',
 'DIGIZOOM': 'None',
 'EXIF': "36864:b'0210'\r\n37121:b'\\x01\\x02\\x03\\x00'\r\n37377:(50, 10)\r\n36867:1999:01:31 15:39:48\r\n36868:1999:01:31 15:39:48\r\n37378:(35, 10)\r\n37380:(0, 100)\r\n37381:(36, 10)\r\n37383:1\r\n37384:0\r\n37385:1\r\n37386:(120, 10)\r\n40961:1\r\n40962:1280\r\n40965:1284\r\n271:EASTMAN KODAK COMPANY\r\n40963:960\r\n272:KODAK DC240 ZOOM DIGITAL CAMERA\r\n274:1\r\n531:1\r\n41493:(140, 1)\r\n41495:2\r\n33432:KODAK DC240 ZOOM DIGITAL CAMERA \r\n41728:b'\\x03'\r\n33434:(1, 30)\r\n282:(192, 1)\r\n283:(192, 1)\r\n33437:(34, 10)\r\n41729:b'\\x01'\r\n296:2\r\n34665:250\r\n40960:b'0100'\r\n"}

In [93]:
getFileCreationTime('Y:/server/EFFIGY_PICS2/Zdjecia/DCP_0474.JPG')

'2020/04/16 18:16:55'

In [94]:
size = os.path.getsize("Y:/server/EFFIGY_PICS2/Zdjecia/IMG_0132noExif.jpg")
print("size", size)

size 919


In [95]:
def insertMediaAttributes (width, height, camera, model, dttaken, dtmodify,\
                          fstop, exposure, orientation, flash, shutter, aperture,\
                          isospeed,  digizoom,  exif,  media_id ,media_key):
    logger.info("Saving Attributes" + media_key)
    #print(key,file_name,media_type,folder_id,path, parent_folder_id,parent_folder,attrib_id,ext)
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "insert into attrib values (null,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        #print (ssql)
        mycursor.execute(ssql, ( width, height, camera, model, dttaken, dtmodify,\
                          fstop,  exposure, orientation, flash, shutter, aperture,\
                          isospeed, digizoom, exif, media_id, media_key, getDt()))
        mydb.commit()
        #get newly created ID
        newid= getID("attrib","attrib_id", "media_id",media_id)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "Folder Completed OK!"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("Folder check DB Closed")
        return (newid)

In [96]:
def ProcessAttributes():
    img_set = getImages()
    for media_id,path,fname, media_key, media_type_id in img_set:
        pth = fixSlashes(path +"/"+fname)
        #print(pth)
        if media_type_id == 'P':
            exif=getImageExifAttributes(pth,media_type_id)
            size = os.path.getsize(pth)
            #print("size", size)
            if size > MIN_IMAGE_SIZE:
                idA = insertMediaAttributes(exif["WIDTH"],exif["HEIGHT"],exif["CAMERA"],exif["MODEL"],exif["DTTAKEN"],exif["DTMODIFY"],exif["FSTOP"],exif["EXPOSURE"],exif["ORIENTATION"],exif["FLASH"],exif["SHUTTER"],exif["APERTURE"],exif["ISOSPEED"],exif["DIGIZOOM"],exif["EXIF"], media_id ,media_key)
            #attrib_id,  width, height,  camera, model,  dttaken,  dtmodify,  fstop,  exposure,  orientation,  flash,  shutter,  aperture,  isospeed,  digizoom,  exif,  media_id ,media_key,'


In [97]:
ProcessAttributes()

Y:/server/EFFIGY_PICS/FILMY_MAMY_20121006/20101105_CUBA_ZDJECIA/100OLYMP/PA110040.JPG
[(8040,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20060710_KUMPLE_ELEKTRYK/PC290069--.jpg
[(8041,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20110820 Eric Pilka/PICT9043.JPG
[(8042,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20070319_all/PICT1759.JPG
[(8043,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/Barbican Drive Neighbourhoods BBQ Aug 5 2002/20020701 Neghbourhoog Barbican Touch Ups/DSCN0688.JPG
[(8044,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20080601 Niagara Falls/PICT3018.JPG
[(8045,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20081228_Mandarin EwaSwieta/PICT4107.JPG
[(8046,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20050731 Quebec City/PICT1702.JPG
[(8047,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20090218 Adas Urodziny/PICT4687.JPG
[(8048,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20101101 4062 Barbican Drive Property Pics/DSC_0270.JPG
[(8049,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20090218 Adas Urodziny/PICT4552.JPG
[(

[(8133,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20110311_Mexico/PICT8127.JPG
[(8134,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/2004 Ready First pics/PICT0167.JPG
[(8135,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20080707_Mexico/PICT3271.JPG
[(8136,)]
Y:/server/EFFIGY_PICS/Zdjecia/RODZICE_ZDJECIA/P5221723.JPG
[(8137,)]
Y:/server/EFFIGY_PICS/Zdjecia/RODZICE_ZDJECIA/P1100434.JPG
[(8138,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20090614_EWA_Camera/IMG_0755.jpg
[(8139,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20050512 Dom Jola CNtower rodzinne/PICT1028.JPG
[(8140,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20070319_all/PICT1842.JPG
[(8141,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20081031 Hallowen/PICT3969.JPG
[(8142,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20100815 Wsaga_AwendaSoubleBeach/PICT7543.JPG
[(8143,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20090706_ Wycieczka_Lake Erie_Turkey_Point/PICT5579.JPG
[(8144,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20101101 4062 Barbican Drive Property Pics/DSC_0191.JPG

[(8228,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20110822_ERIC_CAMERA/P8181223.JPG
[(8229,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20060208 Baby boy/PICT0358.JPG
[(8230,)]
Y:/server/EFFIGY_PICS/FILMY_MAMY_20121006/20101105_CUBA_ZDJECIA/100OLYMP/PA140091.JPG
[(8231,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20051022Dominicana/PICT0659.JPG
[(8232,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/1_DUPLICATES MAYBE_ZDJECIA_part_polska/20070319_all/PICT1750.JPG
[(8233,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20060208 Baby boy/PICT0499.JPG
[(8234,)]
Y:/server/EFFIGY_PICS/Zdjecia/RODZICE_ZDJECIA/P4250727.JPG
[(8235,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20100628 Adam Pilka/PICT6915.JPG
[(8236,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20050522 Jola Niagara/PICT1254.JPG
[(8237,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20060208 Baby boy/PICT0490.JPG
[(8238,)]
Y:/server/EFFIGY_PICS/Zdjecia/RODZICE_ZDJECIA/P4241536.JPG
[(8239,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20050911-rodzinne/PICT2085.JPG
[(8240,)]
Y:/s

[(8323,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20060620-Polska/PICT0918.JPG
[(8324,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20050801 - Pinery/PICT1920.JPG
[(8325,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20090706_ Wycieczka_Lake Erie_Turkey_Point/PICT5359.JPG
[(8326,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20071010_Zosia_HighPark_Chomik/PICT2335.JPG
[(8327,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20080828 Cottage_mojeUrodziny/PICT3805.JPG
[(8328,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20050724 Urodziny prosiak/PICT1571.JPG
[(8329,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20100628 Adam Pilka/PICT7063.JPG
[(8330,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20100815 Wsaga_AwendaSoubleBeach/PICT7234.JPG
[(8331,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20110822_ERIC_CAMERA/P7241154.JPG
[(8332,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20100703 Polska_Sylwia_Dziadek/PICT7089.JPG
[(8333,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20081228_Mandarin EwaSwieta/PICT4162.JPG
[(8334,)]
Y:/server/EFFIGY_PICS/Zdjecia

[(8420,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/1_Rodzinne _DUPLICATES/PICT1590.JPG
[(8421,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20110822_ERIC_CAMERA/PC280204.JPG
[(8422,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20071231Sylwester2007/PICT1989.JPG
[(8423,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20090706_ Wycieczka_Lake Erie_Turkey_Point/PICT5387.JPG
[(8424,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20101101 4062 Barbican Drive Property Pics/DSC_0248.JPG
[(8425,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/2004 Ready First pics/PICT0140.JPG
[(8426,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/1_DUPLICATES MAYBE_ZDJECIA_part_polska/20070319_all/PICT1386.JPG
[(8427,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20080511 Komunia Eryka/PICT2872.JPG
[(8428,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20100725 Ogrodek/PICT7168.JPG
[(8429,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/1_DUPLICATES MAYBE_ZDJECIA_part_polska/babcia2/PICT0639.JPG
[(8430,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20060604ChrzcinyAdam/PICT0677.JPG
[(8

...

[(29801,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20100720 AdamPilka/PICT7129.JPG
[(29802,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20070410WielkanocJustynaAdam/PICT2006.JPG
[(29803,)]
Y:/server/EFFIGY_PICS/Zdjecia/RODZICE_ZDJECIA/P3221373.JPG
[(29804,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/WEB/Dominicana/images/pict0756.jpg
[(29805,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20090524_Niagra_Falls_Ewa/PICT5161.JPG
[(29806,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20080511 Komunia Eryka/PICT2657.JPG
[(29807,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/WEB/zdjecia/thumbnails/pict0509.jpg
[(29808,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/1_DUPLICATES MAYBE_ZDJECIA_part_polska/Ewa/PICT3707.JPG
[(29809,)]
Y:/server/EFFIGY_PICS/Zdjecia/RODZICE_ZDJECIA/P3050484.JPG
[(29810,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20100628 Adam Pilka/PICT6756.JPG
[(29811,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20100905 WoodsCottage_house_Chrzest/PICT7762.JPG
[(29812,)]
Y:/server/EFFIGY_PICS/Zdjecia/RODZICE_ZDJECIA/PB020204.

[(29895,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20060205CYRK_Swieta/PICT0580.JPG
[(29896,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20080601Z zosi kamery/IMG_0230.jpg
[(29897,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/WEB/Holquin/images/pict0151.jpg
[(29898,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20050522 Jola Niagara/PICT1261.JPG
[(29899,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/WEB/zdjecia/thumbnails/pict0127.jpg
[(29900,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20110527_Sylvia_40_Birthday/PICT8429.JPG
[(29901,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20090524_Niagra_Falls_Ewa/PICT5138.JPG
[(29902,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20100715_NowaCamera_NiagaraRoma/IMG_0252.JPG
[(29903,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/1_Rodzinne _DUPLICATES/Zdjecia/forNK/WasagaBeach/0-0-Copy of PICT3620.JPG
[(29904,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20080201Adas_ZosiaMamaUrodziny/PICT2539.JPG
[(29905,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/1_DUPLICATES MAYBE_ZDJECIA_part_polska/Zdjecia/cut/P

[(29986,)]
Y:/server/EFFIGY_PICS/Zdjecia/RODZICE_ZDJECIA/PA120038.JPG
[(29987,)]
Y:/server/EFFIGY_PICS/Zdjecia/RODZICE_ZDJECIA/P5201697.JPG
[(29988,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20090524_Niagra_Falls_Ewa/PICT5009.JPG
[(29989,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20090614_EWA_Camera/IMG_1113.jpg
[(29990,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20100628 Adam Pilka/PICT6599.JPG
[(29991,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/1_DUPLICATES MAYBE_ZDJECIA_part_polska/20070319_all/PICT1699.JPG
[(29992,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20080601 Niagara Falls/PICT3031.JPG
[(29993,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20070319_all/PICT1718.JPG
[(29994,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/WEB/Niagra/thumbnails/pict1312.jpg
[(29995,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/1_Rodzinne _DUPLICATES/PICT1287.JPG
[(29996,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/WEB/Niagra/thumbnails/pict1195.jpg
[(29997,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/1_DUPLICATES MAYBE_ZDJECIA_part_polska

[(30076,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20101101 HOUSE_4062_BARBICAN/PICT3673.JPG
[(30077,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/WEB/wasik/thumbnails/pict0459.jpg
[(30078,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/2004 Ready First pics/PICT0150.JPG
[(30079,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20051015-wujekFarma/PICT2145.JPG
[(30080,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/WEB/Pinery/Pinery 2005/thumbnails/pict1843.jpg
[(30081,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20030501_FIDELITY_STATEK/R003-007.JPG
[(30082,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20080828 Cottage_mojeUrodziny/PICT3804.JPG
[(30083,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20070319_all/PICT1431.JPG
[(30084,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/2070810_AdamNana_Cleinbourg/PICT2304.JPG
[(30085,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20080828 Cottage_mojeUrodziny/PICT3747.JPG
[(30086,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20051030-ZOO/PICT2212.JPG
[(30087,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/1_DUPLI

[(30170,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20100725 Ogrodek/PICT7175.JPG
[(30171,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20110820 Eric Pilka/PICT8918.JPG
[(30172,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/SylwiaRodzinne/Orginaly/ChrzcinyPatrycja.jpg
[(30173,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/1_Rodzinne _DUPLICATES/PICT1904.JPG
[(30174,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20101101 4062 Barbican Drive Property Pics/DSC01128.JPG
[(30175,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20051015-wujekFarma/PICT2069.JPG
[(30176,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20080601 Niagara Falls/PICT3053.JPG
[(30177,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/1_DUPLICATES MAYBE_ZDJECIA_part_polska/20090327 Punta Cana/PICT4625.JPG
[(30178,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20070319_all/PICT1289.JPG
[(30179,)]
Y:/server/EFFIGY_PICS/Zdjecia/Minolta/20050905 Rob's Cottage Summer 2005/20060707 Cottage Collection Can friends/Mike's Pictures 001.jpg
[(30180,)]
Y:/server/EFFIGY_PICS/Zdjecia2008_II/20

In [562]:

img_set = getImages()
for media_id,path,fname, media_key, media_type_id in img_set:
    pth = fixSlashes(path +"/"+fname)
    print(pth)
    if media_type_id == 'P':
        exif=getImageExifAttributes(pth)
        print (len(exif["EXIF"]))

    

In [563]:
exif = getImageExifAttributes("Y:/server/EFFIGY_PICS2\\Zdjecia/other/IMG_0025.jpg")
exif["WIDTH"]
#print(exif)

#for e in exif:
#    print(e)
print(str(exif["APERTURE"]))
print(exif["APERTURE"])
print(convertTuple(exif["APERTURE"]))
#idA = inserMediaAttributes(exif["WIDTH"],exif["HEIGHT"],exif["CAMERA"],exif["MODEL"],exif["DTTAKEN"],exif["DTMODIFY"],exif["FSTOP"],exif["EXPOSURE"],exif["ORIENTATION"],exif["FLASH"],exif["SHUTTER"],exif["APERTURE"],exif["ISOSPEED"],exif["DIGIZOOM"],"","1" ,"1")
             

TypeError: getImageExifAttributes() missing 1 required positional argument: 'ext'

In [564]:
path = "C:/GIT_REPOSITORY/REPO/effigy/imageLib/test.jpg"
from PIL import Image
# Read image 
img = Image.open(path) 
  
# Output Images 
img.show() 

# prints format of image 
print(img.format) 

# prints mode of image 
print(img.mode) 

MPO
RGB


In [565]:

#path = "C:/GIT_REPOSITORY/REPO/effigy/imageLib/geotag.jpg"
path= "Y:/server/EFFIGY_PICS2/Zdjecia/other/IMG_0131.jpg"
# Python program to read 
# image using PIL module 
  
# importing PIL 
from PIL import Image 
  
# Read image 
img = Image.open(path) 
  
# Output Images 
img.show() 
  
# prints format of image 
print(img.format) 
  
# prints mode of image 
print(img.mode) 
dic = img._getexif()
width = dic.get(40963)
height = dic.get(40962)
camera = dic.get(271)
model = dic.get(272)
dttaken = dic.get(36868)
dtmod= dic.get(36867)
fstop= dic.get(42036)
exposure = dic.get(40960) 
orientation=dic.get(274)
flash = dic.get(7385)
shutter = dic.get(37377)
apperture=dic.get(37378) 
isospeed=dic.get(34855) 
digzoom=dic.get(41988)
exif=""


JPEG
RGB
